In [1]:
import pandas as pd
import numpy as np
import awswrangler as wr
import os
from sklearn.preprocessing import MultiLabelBinarizer

#os.chdir('..')
#from src.preprocessing import preprocess

pd.options.display.max_columns = 999

%matplotlib inline

In [2]:
q = """
select *
from coe_analytics_tmp.ao_cross3_vehjoin8;
"""

data = wr.athena.read_sql_query(q, database="bdax_acselx")

In [3]:
import pandas as pd 
import numpy as np 

import awswrangler as wr
import pyarrow.compat as compat
import pyarrow.parquet as pq
import s3fs

import lightgbm as lgb
import pickle 

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pyarrow/compat.py:25: FutureWarning: pyarrow.compat has been deprecated and will be removed in a future release
  "future release", FutureWarning)


In [4]:
variables = [
 'avg_beneficio_pps', 'scoreingreso', 'tipotrabajador', 'dem_edad',
 #'e_giro',#'last_univ', #'last_grado',
 #'polizas_vencidas', #'act_polizas_vencidas', #'meses_ult_retraso_cob',
 'last_prima_tot_miscli', 'avg_prima_tot_miscli', 'PRIMA_PROM_GRUPOS', 'PRIMA_PROM_RIESGOS', 'ratio_inc_prima_tot_miscli',

 #'dem_grado_instruccion',#'rcc_num_vehic_sbs_6','dem_ocupacion', 'ESTRATO', 'dem_dep_agrup',
 'dem_lima_agrup', 'dem_cono_agrup', 'dem_nse_rimac',
 'rcc_cal_gral', 'rcc_num_vehic_sbs',  'rcc_linea_tcmax', 'rcc_linea_tcmax_var6',
 'veh_marca', 'veh_modelo',  'veh_valor_act', 'antiguedad_veh',
 #'veh_clase',  ## RARO
 'num_grupos', 'num_riesgos',
 'VEH_firstgrupos', 'EPS_grupos', 'SOA_grupos', 'VEH_var', 'RIESGOS GENERALES_riesgos',
 'months_ant_miscli2', 'num_meses_miscli', 'target', 'periodo_target', 'cuc', 'placa', 'jerarquia'
]

In [5]:
def replace_otro(df, col, rep = 'OTRO', thresh = 0.01):
    aux = df[col].astype('object').value_counts(normalize=True)
    df.loc[~df[col].isin(aux[aux>=thresh].index),col] = 'OTRO'
    
    return df

In [6]:
# combo_riesgos Binarizar
def encoding_mlbin(df, var, sufix):
    
    tmp = df[var].fillna('ND_'+var).str.split('/')
    mlb = MultiLabelBinarizer()
    aux = pd.DataFrame(mlb.fit_transform(tmp), columns=mlb.classes_, index=df.index)
    aux['num'] = aux.sum(axis = 1)
    
    if(sufix): aux = aux.add_suffix('_'+sufix)
    
    df = pd.concat([df, aux], axis = 1)
    
    return df



In [13]:
def pre_process_data(df):
    
    df['periodo_target'] = df['periodo_target'].astype('int')
    
    # EXCLUIMOS CLIENTES
    prod_bloqueados=['VEHV']
    df = df[~df['grupo_analytics_concat'].str.contains(r'\b(?:{})\b'.format('|'.join(['VEHV']))).fillna(False)]
    df = df[(df['dem_edad']>18) & (df['dem_edad'].notnull())]
    
    df['veh_anhofab'] = df['veh_anhofab'].fillna(2000)
    df = encoding_mlbin(df, 'grupo_analytics_concat', 'grupos')
    df = encoding_mlbin(df, 'combo_riesgos_miscli', 'riesgos')
    df = encoding_mlbin(df, 'combo_canales_miscli', 'canales')
    df = encoding_mlbin(df, 'first_grupo_analytics_concat', 'firstgrupos')

    df['PRIMA_PROM_GRUPOS'] = df['last_prima_tot_miscli'] / df['num_grupos']
    df['PRIMA_PROM_RIESGOS'] = df['last_prima_tot_miscli'] / df['num_riesgos']

    df['ratio_inc_prima_tot_miscli'] =df['last_prima_tot_miscli']  / df['avg_prima_tot_miscli'] 
    
    df['rcc_linea_tcmax_var6'] =  df['rcc_linea_tcmax'] - df['rcc_linea_tcmax_6']
    
    df['months_ant_miscli'] = ((pd.to_datetime(df['periodo_target'], format ='%Y%m') - 
                                pd.to_datetime(df['first_periodo'].astype('str'), format ='%Y%m') )/np.timedelta64(1, 'M'))

    df['months_ant_miscli2'] = np.where(df['months_ant_miscli']>36, 36, df['months_ant_miscli'])
    grupos = [col.split('_')[0] for col in df.columns if  '_grupos' in col ]

    for col in grupos:
        if (col+'_grupos' in df.columns) and (col+'_firstgrupos' in df.columns):
            df[col+'_var'] = df[col+'_grupos'] - df[col+'_firstgrupos']
        else:
            print('no esta', col)

    var_exl = ['_firstgrupos',  '_canales',  '_grupos', '_mediopago', '_firstgrupos',  'grupo_analytics_concat', 'combo_riesgos_miscli', 
           'mediopago_concat', 'combo_canales_miscli', 'first_grupo_analytics_concat', 'first_combo_riesgos_miscli', 
           'first_combo_canales_miscli', 'last_dipl_fecha', 'fec_ingreso_adn']

    var_ids = ['cuc_tit', 'periodo_target', 'placa']
    target = ['veh']
    features = list(set(df.columns.tolist()) - set(var_exl + var_ids + target))
    
    df['rcc_linea_tcmax_6']= pd.to_numeric(df['rcc_linea_tcmax_6'], errors='coerce') 
    df['rcc_linea_tcmax']= pd.to_numeric(df['rcc_linea_tcmax'], errors='coerce') 
    df['rcc_num_vehic_sbs']= pd.to_numeric(df['rcc_num_vehic_sbs'], errors='coerce') 
    df['rcc_saldo_tc_sbs']= pd.to_numeric(df['rcc_saldo_tc_sbs'], errors='coerce') 
    df['rcc_linea_tcmax_var6']= pd.to_numeric(df['rcc_linea_tcmax_var6'], errors='coerce') 
    df['scoreingreso'] = pd.to_numeric(df['scoreingreso'], errors='coerce') 
    
    
    df['antiguedad_veh'] = (df['periodo_target']//100 - df['veh_anhofab']).astype(int)
    df['veh_marca'] = df['veh_marca'].str.rstrip('.!? \n\t').str.lstrip('.!? \n\t')
    df['veh_modelo'] = df['veh_modelo'].str.rstrip('.!? \n\t').str.lstrip('.!? \n\t')
    
    df = replace_otro(df = df, col = 'veh_marca', rep = 'OTRO', thresh = 0.002)
    df = replace_otro(df = df, col = 'veh_modelo', rep = 'OTRO', thresh = 0.001)
    
    df = df.rename(columns={'veh': 'target'})
    #dic_uso = {
    #'PARTICULAR': 'PARTICULAR', 'PERSONA': 'PARTICULAR', 'ESCOLAR': 'PARTICULAR', 'PARTICULAR - CAMIONETA RURAL': 'PARTICULAR',
    #'PARTICULAR - STATION WAGON': 'PARTICULAR', 'AUTOMOVIL': 'PARTICULAR',
    #'CARGA/TRANSPORTE': 'CARGA/TRANSPORTE','CARGA/GRUA': 'CARGA/TRANSPORTE', 'CARGA TRANSPORTE': 'CARGA/TRANSPORTE', 
    #'URBANO/INTERURBANO':'URBANO/INTERURBANO', 'URBANO/INTERURBANO - AUTOMOVIL': 'URBANO/INTERURBANO','INTERPROVINCIAL': 'URBANO/INTERURBANO', 
    #'TAXI': 'TAXI', 'TAXI - AUTOMOVIL':'TAXI',
    #'OTROS': 'OTROS', 'nan':'ND'}
    #df['veh_uso_resto'] = df['veh_uso_resto'].astype('str').fillna('ND').map(dic_uso).fillna('OTROS').astype('category')
    
    for col in ['tipotrabajador', 'dem_lima_agrup', 'dem_cono_agrup', 'dem_nse_rimac', 'rcc_cal_gral', 'veh_marca', 'veh_modelo']:
        df[col] = df[col].astype('category')
    
    for col in ['avg_beneficio_pps', 'scoreingreso', 'dem_edad', 'last_prima_tot_miscli', 'avg_prima_tot_miscli', 'PRIMA_PROM_GRUPOS',
                'PRIMA_PROM_RIESGOS', 'ratio_inc_prima_tot_miscli', 'rcc_num_vehic_sbs', 'rcc_linea_tcmax', 'rcc_linea_tcmax_var6',
                'veh_valor_act', 'antiguedad_veh', 'num_grupos', 'num_riesgos', 'VEH_firstgrupos', 'EPS_grupos', 'SOA_grupos', 'VEH_var',                      
                'RIESGOS GENERALES_riesgos', 'months_ant_miscli2', 'num_meses_miscli', 'target']:
        df[col] = df[col].astype('float')
            
    df_output = df.copy()
    
    return df_output[variables]

In [8]:
data.loc[data['first_periodo'].isnull(), 'first_periodo'] = '202102'

In [14]:
data_prec = pre_process_data(data)

no esta ND


In [15]:
wr.s3.to_parquet(
        df=data_prec.reset_index(drop=True),
        path='s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/all/data.parquet',
        mode='overwrite',
        dataset=True,
)


{'paths': ['s3://rimac-analytics-temporal/individuals/Dante/cross-sell-veh/data/prec/all/data.parquet/a22247e00bf245c4942f97b8235d63e4.snappy.parquet'],
 'partitions_values': {}}